In [ ]:
import numpy as np
import math
from math import pi
import time
import random

n=2
N=2**n
N2=4**n
epsilon = 0.01
r_dig = 2 #Rounding place

# Define the matrices
I = np.array([[1, 0], [0, 1]])
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])
T = np.array([[1,0],[0, np.exp((1j*pi)/4)]])

def tensor_product_recursive(paulis, depth):
        if depth == 1:
            return paulis
        else:
            return [np.kron(p, q) for p in paulis for q in tensor_product_recursive(paulis, depth - 1)]

def tensor_product(operators):
    result = operators[0]
    for op in operators[1:]:
        result = np.kron(result, op)
    return result

def trace_product(W_prime, P_out, P_in):
    product = np.matmul(W_prime, np.matmul(P_out, np.matmul(W_prime.conj().T, P_in)))
    return np.trace(product)

def matrix_product_recursive(S, depth):
        if depth == 1:
            return S
        else:
            return [np.matmul(p, q) for p in S for q in matrix_product_recursive(S, depth - 1)]

def matrix_neg_check(A, B, N):
    for i in range(N):
        for j in range(N):
            if A[i][j] != -B[i][j]:
                return False
    return True

def matrix_eq_check(A,B,N): #Returns 1 if equal
    for i in range(N):
      if eq == 0:
        break
      for j in range(N):
        #if (A[i][j] - B[i][j] < 10**-6) or (B[i][j] - A[i][j] < 10**-6):
        if (A[i][j] - B[i][j] == 0):
          eq = 1
        else:
          eq = 0
          break
    return eq

def generate_pauli_n(n):
    """Generate the set of Pauli matrices for n qubits."""
    # Base Pauli matrices
    I = np.array([[1, 0], [0, 1]])
    X = np.array([[0, 1], [1, 0]])
    Y = np.array([[0, -1j], [1j, 0]])
    Z = np.array([[1, 0], [0, -1]])
    T = np.array([[1,0],[0, np.exp((1j*pi)/4)]])
    paulis = [I, X, Y, Z]

    return tensor_product_recursive(paulis, n)

def pauli_string_to_matrix(pauli_string):
    # Convert a Pauli string (e.g., 'IXY') to its matrix representation
    # You might need to implement this function based on your requirements

    pauli_dict = {'I': I, 'X': X, 'Y': Y, 'Z': Z}
    matrix = np.kron(pauli_dict[pauli_string[0]], pauli_dict[pauli_string[1]])

    for p in pauli_string[2:]:
      matrix = np.kron(matrix, pauli_dict[pauli_string[2]])

    return matrix

pauli_I = [I]
i_tensored = tensor_product_recursive(pauli_I, n)[0]
pauli_n = generate_pauli_n(n)

def CLIFF_TEST(W_prime): #Returns 1 if Clifford

  i = 0
  cliff = 1
  for P in pauli_n:
    result = np.matmul(W_prime, P)
    tr_result = np.abs(np.trace(result) / N)
    if (tr_result != 0) and (i == 0):
      if tr_result == 1:
        break
      else:
        prev_amp = tr_result
        i = i+1
    if (tr_result != 0) and (i != 0):
      if tr_result != prev_amp:
        print("Not Clifford after amplitude test.")
        cliff = 0
        return 0

  for P_out in pauli_n:
    if cliff == 0:
      print("Exiting outer loop of conjugation test")
      return 0
    if np.allclose(P_out, i_tensored) == True:
      #print("Got I!")
      continue

    for P_in in pauli_n:
      if np.allclose(P_in, i_tensored) == True:
        #print("Got I!")
        continue
      val = abs(trace_product(W_prime, P_out, P_in)) / N
      if val == 1:
        break
      if (val != 1) and (val != 0):
        print("Not Clifford after conjugation test.")
        cliff = 0
        break

  if cliff == 1:
    print("Passed Clifford test")
    return 1

b_conj = 1 - 4 * epsilon**2 + 2 * epsilon**4
print("b_conj = ",b_conj)
#b_conj = round(b_conj,r_dig)
print("b_conj after rounding  = ",b_conj)

l_conj = 2 * epsilon
print("l_conj = ",l_conj)
#l_conj = round(l_conj,r_dig)
print("l_conj after rounding = ",l_conj)

def A_CONJ(W_prime, epsilon):

    p = 1

    for P_out in pauli_n:
        if np.allclose(P_out, i_tensored) == True:
            #print("Got I!")
            continue
        if p == 1:
            p = 0

        prod_out = np.matmul(W_prime, np.matmul(P_out, W_prime.conj().T))
        for P_in in pauli_n:
            if np.allclose(P_in, i_tensored) == True:
                #print("Got I!")
                continue
            prod_in = np.matmul(prod_out, P_in)
            val0 = abs(np.trace(prod_in)) / N
            val = round(val0,r_dig)
            print("val0,val, b_conj,l_conj = ",val0,val, b_conj,l_conj)
            #b_conj = 1 - 4 * epsilon**2 + 2 * epsilon**4

            if b_conj <= val <= 1:
                p += 1
                print("satisfies 1st ineq: p = ",p)
                if p > 1:
                    return "NO"
            else:
                print("notsatisfies 1st ineq")

            if l_conj < val < b_conj:
                print("satisfies 2nd ineq")
                return "NO"

    return "YES"

LB1 = []
UB1 = []
UB0 = []

for M in range(1,N2+1):
  lb_1 = (1 - epsilon**2) / np.sqrt(M) - np.sqrt(M * (2 * epsilon**2 - epsilon**4))
  LB1.append(lb_1)
  ub_1 = (1 / np.sqrt(M)) + np.sqrt(M * (2 * epsilon**2 - epsilon**4))
  UB1.append(ub_1)
  ub_0 = np.sqrt(M * (2 * epsilon**2 - epsilon**4))
  UB0.append(ub_0)

print("LB1 = ",LB1)
print("UB1 = ",UB1)
print("UB0 = ",UB0)

#for M in range(0,N2):
  #LB1[M] = round(LB1[M],r_dig)
  #UB1[M] = round(UB1[M],r_dig)
  #UB0[M] = round(UB0[M],r_dig)

print("LB1 after rounding = ",LB1)
print("UB1 after rounding = ",UB1)
print("UB0 after rounding = ",UB0)

def A_DECIDE(W,U_tilde, epsilon):

  W_prime = np.matmul(np.conj(W.T), U_tilde)
  #tr_Wprime = np.abs(np.trace(W_prime) / N)
  #dist = math.sqrt(1-tr_Wprime)
  S_c = []

  for P in pauli_n:
    result = np.matmul(W_prime, P)
    tr_result = np.abs(np.trace(result) / N)
    tr_result = round(tr_result,r_dig)
    S_c.append(tr_result)

  S_c.sort(reverse=True)

  for M in range(1,4**n+1):
      S_1 = S_c[:M]
      S_0 = S_c[M:]
      #lb1 = (1 - epsilon**2) / np.sqrt(M) - np.sqrt(M * (2 * epsilon**2 - epsilon**4))
      #ub1 = (1 / np.sqrt(M)) + np.sqrt(M * (2 * epsilon**2 - epsilon**4))
      #ub0 = np.sqrt(M * (2 * epsilon**2 - epsilon**4))
      amp = 1
      for x in S_1:
        if LB1[M-1] <= x <= UB1[M-1]:
          amp = 1
        else:
          amp = 0
          break
        if amp == 1:
          for x in S_0:
            if 0 <= x <= UB0[M-1]:
              amp = 1
            else:
              amp = 0
              break
        if amp == 1:
          print("Conj begin")
          result = A_CONJ(W_prime, epsilon)
          if result == "YES":
            execTime = time.time()-start_time
            #print("Fin Utilde",U_tilde)
            #print("Fin S_c",S_c)
            #print("Fin M",M)
            #print("Fin S_1",S_1)
            #print("Fin S_0",S_0)
            return 2 #Passed both tests
          else:
            return 1 # Passed amp, did not pass conj

  return 0


b_conj =  0.99960002
b_conj after rounding  =  0.99960002
l_conj =  0.02
l_conj after rounding =  0.02
LB1 =  [0.9857582179340791, 0.687036570514679, 0.5527982491149657, 0.4716664358681584, 0.41554688811802154, 0.37356731551966854, 0.34051103812022654, 0.31351903526671476, 0.2908746538022375, 0.271475901738205, 0.25457820846366946, 0.23965769698587097, 0.2263334427376887, 0.21432081245913323, 0.2034021834311803, 0.19340787173631668]
UB1 =  [1.0141417820659209, 0.7271062811802973, 0.601844554237367, 0.5282835641318416, 0.4788355815223443, 0.4428884405790112, 0.41538011145092696, 0.3935523905807734, 0.3757586795310958, 0.36094800751886924, 0.34841432955740004, 0.3376637046902954, 0.32833901847772917, 0.32017494524152434, 0.3129697761741672, 0.3065671282636833]
UB0 =  [0.01414178206592083, 0.019999499993749843, 0.02449428504774124, 0.02828356413184166, 0.03162198602238639, 0.03464015011514817, 0.037415638441699754, 0.03999899998749969, 0.04242534619776249, 0.04472024150203127, 0.046902984

In [ ]:
S1 = []
S2 = []
S3 = []
S4 = []
S5 = []
S6 = []
S7 = []
S8 = []
S9 = []
S10 = []

cs_gen = [('IX', 'XI'), ('IX', 'YI'), ('IX', 'ZI'), ('IY', 'XI'), ('IY', 'YI'), ('IY', 'ZI'), ('IZ', 'XI'), ('IZ', 'YI'), ('IZ', 'ZI'), ('XX', 'YY'), ('XX', 'YZ'), ('XY', 'YX'), ('XY', 'YZ'), ('XZ', 'YX'), ('XZ', 'YY')]
cs_gen_t = []
beta_0 = (3 + 1j)/4
beta_1 = (1 - 1j)/4
for l in range(len(cs_gen)):
  P1 = pauli_string_to_matrix(cs_gen[l][0])
  P2 = pauli_string_to_matrix(cs_gen[l][1])
  G_P12 = beta_0*i_tensored + beta_1*(P1 + P2 - np.matmul(P1, P2))
  cs_gen_t.append(G_P12)
  #print("l,P1,P2,cs_gen_t = ",l,P1,P2,cs_gen_t[l])

print("size of cs_gen_t = ",len(cs_gen_t))
size_gcs = len(cs_gen)
print("size_gcs = ",size_gcs)

for l1 in range(size_gcs):
  prod1 = cs_gen_t[l1]
  S1.append(prod1)
  for l2 in range(size_gcs):
    if l2 == l1:
      continue
    prod2 = np.matmul(prod1,cs_gen_t[l2])
    S2.append(prod2)
    for l3 in range(size_gcs):
      if l3 == l2:
        continue
      prod3 = np.matmul(prod2,cs_gen_t[l3])
      S3.append(prod3)
      for l4 in range(size_gcs):
        if l4 == l3:
          continue
        prod4 = np.matmul(prod3,cs_gen_t[l4])
        S4.append(prod4)
        for l5 in range(size_gcs):
          if l5 == l4:
            continue
          prod5 = np.matmul(prod4,cs_gen_t[l5])
          S5.append(prod5)
          for l6 in range(size_gcs):
            if l6 == l5:
              continue
            prod6 = np.matmul(prod5,cs_gen_t[l6])
            S6.append(prod6)
            #for l7 in range(size_gcs):
              #if l7 == l6:
                #continue
              #prod = np.matmul(prod,cs_gen_t[l7])
              #S7.append(prod)

size_S1 = len(S1)
size_S2 = len(S2)
size_S3 = len(S3)
size_S4 = len(S4)
size_S5 = len(S5)
size_S6 = len(S6)
#size_l7 = len(S7)

print("Size of S1 = ",size_S1)
print("Size of S2 = ",size_S2)
print("Size of S3 = ",size_S3)
print("Size of S4 = ",size_S4)
print("Size of S5 = ",size_S5)
print("Size of S6 = ",size_S6)
#print("Size of S7 = ",size_S7)

size of cs_gen_t =  15
size_gcs =  15
Size of S1 =  15
Size of S2 =  210
Size of S3 =  2940
Size of S4 =  41160
Size of S5 =  576240
Size of S6 =  8067360


In [ ]:
#W = np.array([[1,0],[0, np.exp((1j*pi)/4)]])  #T gate
theta_k = pow(2,2)  #k=0 : Z, k=1: S, k=2 : T
W = np.array([[1,0,0,0],[0, np.exp((1j*pi)/theta_k),0,0],[0,0,1,0],[0,0,0,np.exp((1j*pi)/theta_k)]]) # Example 2x2 unitary matrix (identity matrix)
print("W, epsilon = ",W, epsilon)
#epsilon = 0.05

cliff_result = CLIFF_TEST(W)
if cliff_result == 1:
  print("Found Clifford")
else:
  print("Not Clifford")



W, epsilon =  [[1.        +0.j         0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j 0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         1.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.70710678+0.70710678j]] 0.01
Not Clifford after amplitude test.
Not Clifford


In [ ]:
succ = 0
N0 = pow(10,6)
S6_temp = []
j1 = 0

for j in range(823144,8067360):
  S6_temp.append(S6[j])

start_time = time.time()

for i1 in range(N0):
  print("i1 = ",i1)
  if succ == 1:
    break
  indx = np.random.randint(0,len(S6_temp))
  U_tilde = S6_temp[indx]
  S6_temp.pop(indx)
  #for i2 in range(size_S3):
    #print("i2_prime = ",i2_prime)
    #U_tilde = np.matmul(U_1, S3[i2])
  #W_prime = np.matmul(np.conj(W.T), U_tilde)
  #tr_Wprime = np.abs(np.trace(W_prime) / N)
  #dist = math.sqrt(1-tr_Wprime)
  result = A_DECIDE(W,U_tilde, epsilon)
  if result == "YES":
    print("YES :")
    succ = succ+1
    break
  #else:
    #print("NO, i1, dist = ",i1,dist)

if succ == 0:
  print("NO")
print("succ = ",succ)

execTime = time.time()-start_time
print("Implementation time = ",execTime)











In [ ]:
#------3rd nesting, random

N1 = 5*pow(10,2)
N2 = 5*pow(10,1)
N3 = pow(10,2)
i = 0
succ = 0
S6_temp = []
for i in range(len(S6)):
  S6_temp.append(S6[i])

print("Size of S6_temp = ",len(S6_temp))

start_time = time.time()
for i1 in range(N1):
  print("i1 = ",i1)
  if succ > 0:
    break
  indx = np.random.randint(0,len(S6_temp))
  U_1 = S6_temp[indx]
  S6_temp.pop(indx)
  S6_temp2 = []
  for i in range(len(S6)):
    S6_temp2.append(S6[i])
  for i2 in range(N2):
    if succ > 0:
      break
    indx = np.random.randint(0,len(S6_temp2))
    U_2 = np.matmul(U_1,S6_temp2[indx])
    S6_temp2.pop(indx)

    S5_temp = []
    for i in range(len(S5)):
      S5_temp.append(S5[i])
    for i3 in range(N3):
      indx = np.random.randint(0,len(S5_temp))
      U_3 = np.matmul(U_2,S5_temp[indx])
      S5_temp.pop(indx)
      #indx = np.random.randint(0,  len(S5_temp))
      #prod = np.matmul(U_3,S5_temp[indx])
      #S5_temp.pop(indx)

      ell4 = 16 #4 last
      prod = U_3
      i4 = 0
      indx_prev = -1
      while i4 < ell4:
        #print("i4 = ",i4)
        indx = np.random.randint(0,size_S1)
        if indx != indx_prev:
          prod = np.matmul(prod, S1[indx])
          indx_prev = indx
          i4 = i4+1

      U_tilde = prod
      result = A_DECIDE(W,U_tilde,epsilon)
      #print("results = ",result)
      if result == 2:
        succ = succ+1
        print("YES")
        break


execTime = time.time()-start_time
print("succ = ",succ)
print("Implementation time = ",execTime)

Size of S6_temp =  8067360
i1 =  0
Conj begin
val0,val, b_conj,l_conj =  0.26317301429693646 0.26 0.99960002 0.02
notsatisfies 1st ineq
satisfies 2nd ineq
i1 =  1
Conj begin
val0,val, b_conj,l_conj =  0.031126627737053554 0.03 0.99960002 0.02
notsatisfies 1st ineq
satisfies 2nd ineq
i1 =  2
i1 =  3
i1 =  4
i1 =  5
Conj begin
val0,val, b_conj,l_conj =  0.22894147037693377 0.23 0.99960002 0.02
notsatisfies 1st ineq
satisfies 2nd ineq
Conj begin
val0,val, b_conj,l_conj =  0.142735528691358 0.14 0.99960002 0.02
notsatisfies 1st ineq
satisfies 2nd ineq
i1 =  6
i1 =  7
i1 =  8
i1 =  9
i1 =  10
i1 =  11
Conj begin
val0,val, b_conj,l_conj =  0.14736527570658264 0.15 0.99960002 0.02
notsatisfies 1st ineq
satisfies 2nd ineq
i1 =  12
i1 =  13
Conj begin
val0,val, b_conj,l_conj =  0.016961240539936112 0.02 0.99960002 0.02
notsatisfies 1st ineq
val0,val, b_conj,l_conj =  0.21015969671556964 0.21 0.99960002 0.02
notsatisfies 1st ineq
satisfies 2nd ineq
i1 =  14
i1 =  15
i1 =  16
i1 =  17
Conj begin


In [ ]:
#-----------------Nesting and expand only along probable ones -----------------------------

N1 = 5*pow(10,2)
N2 = 5*pow(10,1)
N3 = pow(10,2)
i = 0
succ = 0
S6_temp = []
for i in range(len(S6)):
  S6_temp.append(S6[i])

print("Size of S6_temp = ",len(S6_temp))

start_time = time.time()
for i1 in range(N1):
  print("i1 = ",i1)
  if succ > 0:
    break
  indx = np.random.randint(0,len(S6_temp))
  U_1 = S6_temp[indx]
  S6_temp.pop(indx)
  S6_temp2 = []
  for i in range(len(S6)):
    S6_temp2.append(S6[i])
  for i2 in range(N2):
    if succ > 0:
      break
    indx = np.random.randint(0,len(S6_temp2))
    U_2 = np.matmul(U_1,S6_temp2[indx])
    S6_temp2.pop(indx)

    S5_temp = []
    for i in range(len(S5)):
      S5_temp.append(S5[i])
    for i3 in range(N3):
      if succ > 0:
        break
      indx = np.random.randint(0,len(S5_temp))
      U_3 = np.matmul(U_2,S5_temp[indx])
      S5_temp.pop(indx)

      result = A_DECIDE(W,U_3,epsilon)
      if result == 2:
        succ = succ+1
        print("YES")
        break
      if result == 1:
        prod = U_3
        N4 = 10
        for i4 in range(N4):
          indx = np.random.randint(0,size_S1)
          prod = np.matmul(prod,S1[indx])
          result = A_DECIDE(W,prod,epsilon)
          print("result4 = ",result)
          if result == 2:
            succ = succ+1
            print("YES")
            break


execTime = time.time()-start_time
print("succ = ",succ)
print("Implementation time = ",execTime)

Size of S6_temp =  8067360
i1 =  0
i1 =  1
i1 =  2
Conj begin
val0,val, b_conj,l_conj =  0.02071601898007465 0.02 0.99960002 0.02
notsatisfies 1st ineq
val0,val, b_conj,l_conj =  0.08286407592029854 0.08 0.99960002 0.02
notsatisfies 1st ineq
satisfies 2nd ineq
result4 =  0
result4 =  0
result4 =  0
result4 =  0
result4 =  0
result4 =  0
result4 =  0
result4 =  0
result4 =  0
result4 =  0
i1 =  3
i1 =  4
i1 =  5
Conj begin
val0,val, b_conj,l_conj =  0.15467960838455724 0.15 0.99960002 0.02
notsatisfies 1st ineq
satisfies 2nd ineq
result4 =  0
result4 =  0
result4 =  0
result4 =  0
result4 =  0
result4 =  0
result4 =  0
result4 =  0
result4 =  0
result4 =  0
i1 =  6
Conj begin
val0,val, b_conj,l_conj =  0.1353446573364876 0.14 0.99960002 0.02
notsatisfies 1st ineq
satisfies 2nd ineq
result4 =  0
result4 =  0
result4 =  0
result4 =  0
result4 =  0
result4 =  0
result4 =  0
result4 =  0
result4 =  0
result4 =  0
i1 =  7
Conj begin
val0,val, b_conj,l_conj =  0.06421965883823139 0.06 0.99960

KeyboardInterrupt: ignored

In [ ]:
#------Random length-----------------------

Niter = pow(10,6)
ell6 = 100  #number of 6count unitaries


start_time = time.time()
succ = 0

for i in range(Niter):
  print("i = ",i)
  prod = i_tensored
  print("prod:I = ",prod)
  indx_prev = -1
  i6 = 0
  while i6 < ell6:
    indx = np.random.randint(0,size_S6)
    if indx != indx_prev:
      #print("indx = ",indx)
      prod = np.matmul(prod,S6[indx])
      indx_prev = indx
      i6 = i6+1
  #indx = np.random.randint(0,size_S1)
  #U_tilde = np.matmul(prod,S1[indx])
  U_tilde = prod
  print("U_tilde = ",U_tilde)
  result = A_DECIDE(W,U_tilde, epsilon)
  if result == "YES":
    print("YES ")
    succ = succ+1
    break
  #else:
    #print("NO, i1, dist = ",i1,dist)


print("succ = ",succ)

execTime = time.time()-start_time
print("Implementation time = ",execTime)




Streaming output truncated to the last 5000 lines.
   0.30348003-0.36420766j]]
i =  1851
prod:I =  [[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]
U_tilde =  [[-0.45699672+0.57549569j  0.27359081+0.34141748j  0.00934592+0.10472714j
   0.23492327+0.44977419j]
 [-0.03516496+0.13080961j  0.03116303-0.01977468j  0.77939366-0.53539432j
  -0.28888163+0.05230635j]
 [-0.49229106-0.37599191j  0.61707797-0.26719474j -0.0099952 -0.1828554j
   0.23707982-0.27268755j]
 [ 0.19352937-0.1429579j   0.33782424-0.49083248j  0.01598416+0.2470867j
  -0.31478789+0.653199j  ]]
i =  1852
prod:I =  [[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]
U_tilde =  [[ 0.095162  +0.13923866j  0.53853508-0.58485839j -0.03756925-0.38280683j
  -0.22942164+0.37268031j]
 [ 0.65637833-0.20528018j  0.40533056+0.01614087j -0.37121541+0.16276113j
   0.2067612 -0.39424851j]
 [-0.30984598-0.16764619j -0.07864841+0.17254926j -0.69298524-0.44836256j
  -0.33732056-0.21186956j]
 [-0.3265065 -0.51755971j  0.34181913+0.22487351j -0.06907082-0

KeyboardInterrupt: ignored